In [53]:
from keras.datasets import mnist, cifar10
# import tensorflow_datasets as tfds
from utils.DOSELM import DOSELM
from utils.DRRN import DRRN
import numpy as np
import torch
import time as t
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

In [2]:
def evaluation_summary(description, predictions, true_labels):
    print("Evaluation for: " + description)
    precision = precision_score(predictions, true_labels, average="macro")
    recall = recall_score(predictions, true_labels, average="macro")
    accuracy = accuracy_score(predictions, true_labels)
    f1 = fbeta_score(predictions, true_labels, beta=1, average="macro")
    print("Classifier '%s' has Acc=%0.3f P=%0.3f R=%0.3f F1=%0.3f" % (description,accuracy,precision,recall,f1))
    print(classification_report(predictions, true_labels, digits=3))
    print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

In [3]:
def scale(X):
    return (X - X.mean())/(X.std()*10)

In [4]:
def splitdata(x, y, splits=2, first=-1):
    length = x.shape[0]

    out = []
    if first == -1:
        splitlen = length // splits
        for i in range(splits):
            out.append((x[splitlen*i:splitlen*(i+1)], y[splitlen*i:splitlen*(i+1)]))
    else:
        splits-= 1
        split1 = int(length*first)
        splitn = (length-split1)//splits
        out.append((x[0:split1], y[0:split1]))
        for i in range(splits):
            a = split1+splitn*i
            b = split1+splitn*(i+1)
            z = (x[a:b], y[a:b])
            out.append(z)
    return out

In [71]:
(train_X, train_y), (test_X, test_y) = cifar10.load_data()
train_X = scale(np.reshape(train_X.ravel(), (-1,3072)))
test_X = scale(np.reshape(test_X.ravel(), (-1,3072)))

y = np.zeros([50000,10])

for i in range(len(y)):
    for j in range(len(y[i])):
        if j == train_y[i]:
            y[i,j] = 1
        else:
            y[i,j] = 0
            
train_y=y

In [195]:
device = torch.device("cpu")

drrn = DRRN(3072, 10, 3500, subnets=1, device=device, r=(0.005, 0.1))

X = torch.from_numpy(train_X).float()[:50000]
y = torch.from_numpy(train_y).float()[:50000]

result = drrn.train(X, y)
tX = torch.from_numpy(test_X).float()
pred = drrn.predict(tX)[:10000]
print(len(test_X))
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

evaluation_summary('DRRN - MNIST', pred_arg, test_y)

10000
Evaluation for: DRRN - MNIST
Classifier 'DRRN - MNIST' has Acc=0.381 P=0.381 R=0.373 F1=0.374
              precision    recall  f1-score   support

         0.0      0.471     0.433     0.451      1089
         1.0      0.468     0.393     0.427      1192
         2.0      0.231     0.306     0.263       755
         3.0      0.194     0.274     0.227       709
         4.0      0.273     0.362     0.311       754
         5.0      0.290     0.318     0.303       913
         6.0      0.465     0.377     0.417      1232
         7.0      0.444     0.427     0.435      1040
         8.0      0.547     0.451     0.494      1213
         9.0      0.431     0.391     0.410      1103

    accuracy                          0.381     10000
   macro avg      0.381     0.373     0.374     10000
weighted avg      0.403     0.381     0.389     10000


Confusion matrix:
 [[471  60  38  22  22  34  23  49 207  74]
 [ 57 468  31  39  26  34  44  56  87 158]
 [105  51 231  67 113  89 153  97  

A current problem with the algorithm, it seems, is that the algorithm can only learn roughly as well as the first representation created by the network. It can maintain its performance accross multiple layers but does not improve it. It is possible that improving the performance of the autoencoder might allow for better results.

In [7]:
torch.manual_seed(0)

device = torch.device("cpu")
doselm = DOSELM(784, 10, [250], device=device)
split_data = splitdata(train_X, train_y, 2)

train_start = t.time()
j = 0

for i in split_data:
    X = torch.from_numpy(i[0]).float()
    y = torch.from_numpy(i[1]).float()
    result = doselm.train(X,y)
    j+=1
    if j==0:
        break
train_end = t.time()

tX = torch.from_numpy(test_X).float()

pred_start = t.time()
pred = doselm.predict(tX)
pred_end = t.time()
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

evaluation_summary('ELM', pred_arg, test_y)

torch.Size([784, 250])


RuntimeError: svd_cpu: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated singular values (error code: 14).

In [ ]:
torch.manual_seed(0)

device = torch.device("cpu")
doselm = DOSELM(784, 10, [1000], device=device)

X = torch.from_numpy(train_X).float()[:60000]
y = torch.from_numpy(train_y).float()[:60000]

result = doselm.train(X, y)
tX = torch.from_numpy(test_X).float()
pred = doselm.predict(tX)
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

evaluation_summary('ELM', pred_arg, test_y)